In [9]:
from bert_serving.client import BertClient
from scipy import spatial
from numpy import argmax
from opencc import OpenCC
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [10]:
cc = OpenCC('t2s')#因為bert目前還是用簡體字做分析，先轉成簡體字
bc = BertClient()#have to run Bert server beforehand
def consine_sim(v1,v2):#餘弦相似性公式
    return 1-spatial.distance.cosine(v1,v2)

# classes = ['房內服務行為','房內看菜單行為','房內點餐行為','服務項目',
#           '訂房行為','詢問健身房地點','詢問健身房開放時間','詢問會議中心地點',
#           '詢問會議中心開放時間','詢問游泳池地點','詢問游泳池開放時間','詢問禮品店地點',
#           '詢問禮品店開放時間','詢問餐廳地點','詢問餐廳開放時間']

classes = ['房內服務行為','詢問健身房地點','詢問餐廳開放時間']
sents_n_labels = [
    ('我要收垃圾，請找房務清潔人員',0),
    ('我要叫房內服務', 0),
    ('找人來幫我掃房間',0),
    ('垃圾太多請來收',0),
    ('房間太髒，需要人來清理',0),
    ('請幫我找管家來',0),
    ('我要換棉被，太薄了，請找管家來',0),
    ('我需要房務人員',0),
    ('請幫我找房務人員過來',0),
    ('房間好髒，需要清掃',0),
    ("健身房的位置在哪裡",1),
    ("你們有健身房嗎",1),
    ("我要去健身房",1),
    ("哪裡可以健身",1),
    ("健身房怎麼去",1),
    ("給我看健身房的位置",1),
    ("健身房在哪裡",1),
    ("健身房在幾樓",1),
    ("請問餐廳什麼時候營業",2),
    ("餐廳營業開放時間是幾點",2),
    ("餐廳開放中嗎",2),
    ("我現在要去餐廳用餐可以嗎",2),
    ("餐廳什麼時候開始",2),
    ("餐廳的開放時間",2),
    ("餐廳現在有開嗎",2),
    ("餐廳的營業時間",2),]


In [5]:
sents =[cc.convert(sent[0]) for sent in sents_n_labels]
print(sents)
labels = [label[1] for label in sents_n_labels]
print(labels)
classes = [cls for cls in classes]
print(classes)
classes_enc = bc.encode(classes)#把句子們轉換成數值向量

['我要收垃圾，请找房务清洁人员', '我要叫房内服务', '找人来帮我扫房间', '垃圾太多请来收', '房间太脏，需要人来清理', '请帮我找管家来', '我要换棉被，太薄了，请找管家来', '我需要房务人员', '请帮我找房务人员过来', '房间好脏，需要清扫', '健身房的位置在哪里', '你们有健身房吗', '我要去健身房', '哪里可以健身', '健身房怎么去', '给我看健身房的位置', '健身房在哪里', '健身房在几楼', '请问餐厅什么时候营业', '餐厅营业开放时间是几点', '餐厅开放中吗', '我现在要去餐厅用餐可以吗', '餐厅什么时候开始', '餐厅的开放时间', '餐厅现在有开吗', '餐厅的营业时间', '我想去吃中餐，有什么推荐', '我想去吃饭', '哪里有餐厅', '让我看看餐厅在哪', '你们餐厅在几楼', '你们有餐厅吗', '我要去餐厅吃饭', '给我餐厅所在地', '你知道餐厅位置吗', '请问用餐地方在哪', '带我去吃东西', '带我去餐厅']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
['房內服務行為', '詢問健身房地點', '詢問餐廳開放時間', '詢問餐廳地點']


In [6]:
def cosine_sim(v1,v2):#餘弦相似性公式
    return 1-spatial.distance.cosine(v1,v2)
def predict_label(v):#v必須是一維向量 因此def another predict_labels
    cos_sim=[]
    for idx,cls in enumerate(classes_enc):
        cos_sim.append(cosine_sim(v,cls))
    return argmax(cos_sim)

def predict_labels(vs):
    op = []
    for v in vs:
        op.append(predict_label(v))
    return op

In [7]:

print("Predict Label:", predict_labels(bc.encode(sents)) )

Predict Label: [0, 0, 1, 3, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3]


In [11]:
cc = OpenCC('t2s')# 繁轉簡
bc = BertClient()# 取得bert服務器資源
def cosine_sim(v1,v2):
    return 1-spatial.distance.cosine(v1,v2)
    


sents = [cc.convert(t[0]) for t in sents_n_labels ]
labels = [t[1] for t in sents_n_labels ]
classes = [cc.convert(s) for s in classes ]

classes_enc = bc.encode(classes)#把句子們轉換成數值向量
sents_enc = bc.encode(sents)

sents_enc_train, sents_enc_test, labels_train, labels_test =  \
train_test_split(sents_enc, labels, test_size=0.33,random_state=42)
cls = SVC()
cls.fit(sents_enc_train, labels_train)
score =cls.score(sents_enc_test, labels_test)
print('Accuracy:',score)


Accuracy: 0.7777777777777778


In [43]:
import urllib.request; #用來建立請求
from sklearn.model_selection import train_test_split
from bert_serving.client import BertClient
from sklearn.svm import SVC

# 引入外部資料集
url = 'https://raw.githubusercontent.com/xxxspy/Chinese_conversation_sentiment/master/sentiment_XS_test.txt'
x = urllib.request.urlopen(url)
x = x.read()
with open('sentiment_XS_test.txt', 'wb') as f: # save
    f.write(x)
    
# 存到本地端後，載入來使用
with open('sentiment_XS_test.txt', 'rb') as f: # load
    x = f.read().decode('utf-8')
x = x.split('\n')
# print(x[0])
# print(x[2])
print(x[1:10])

['negative,AP 好 任性', 'negative,a 甜心 一手 货源 招 代理  这个 骗子', 'negative,っ ╥ ╯ ﹏ ╰ ╥ c 被 土豪 欺负', 'negative,❀ ℋ č 点点  讨厌 不许 亲亲', 'negative,Don t care  大爷 此', 'negative,ee 好 霸道 欺负 老实人', 'negative,E 无语', 'negative,fct 很 讨厌', 'negative,fly 白痴']


In [46]:
labels1, sents1 = [], []
d = { 'negative':0, 'positive':1 }
for line in x[1:]:
    sp = line.split(',')
    if len(sp) != 2:
#         print(sp)
        continue
    pn, text = sp
    label = d[pn.strip()]
    labels1.append(label) 
    sents1.append(text)


In [47]:
bc = BertClient() # 取得bert服務器資源
sents_enc1 = bc.encode(sents1) #把句子們轉換成數值向量

X_train, X_test, Y_train, Y_test =  \
train_test_split(sents_enc1, labels1, test_size=0.33,random_state=36)

In [49]:
module = SVC()
module.fit(X_train,Y_train)
module.score(X_test, Y_test)

0.9185010482180294